In [23]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

Credentials are already in use. The existing account in the session will be replaced.


In [26]:
from collections import OrderedDict
import numpy as np
from qiskit.quantum_info import Pauli
from qiskit.aqua import operators
from qiskit.quantum_info import *
from qiskit.aqua.algorithms.adaptive import QAOA
from qiskit.aqua.components.optimizers import *
from qiskit.aqua import QuantumInstance
import networkx as nx
from qiskit.aqua import get_aer_backend

def sample_most_likely(state_vector):
    if isinstance(state_vector, dict) or isinstance(state_vector, OrderedDict):
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x
def get_qubitops(input):
    w = np.array(input.tolist())
    num_nodes = len(w)
    pauli_list = []
    shift = 0
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            if w[i, j] != 0:
                xp = np.zeros(num_nodes, dtype=np.bool)
                zp = np.zeros(num_nodes, dtype=np.bool)
                zp[i] = True
                zp[j] = True
                pauli_list.append([w[i, j], Pauli(zp, xp)])
                shift += 1
    for i in range(num_nodes):
        degree = np.sum(w[i, :])
        xp = np.zeros(num_nodes, dtype=np.bool)
        zp = np.zeros(num_nodes, dtype=np.bool)
        zp[i] = True
        pauli_list.append([w[i, i], Pauli(zp, xp)])
    return Operator(paulis=pauli_list)

from qiskit import IBMQ
IBMQ.load_account()

def get_backend(backend_str):
    if backend_str == 'qasm_simulator':
        backend = get_aer_backend(backend_str)
    elif backend_str == 'ibmq_16_melbourne' or backend_str == 'ibmqx4':
        backend = IBMQ.get_backend(backend_str)
    else:
        print("Choose a valid backend")
        return
    return backend

def solve_ibmqx_ising_qubo(G, matrix_func, optimizer, p, backend_str='qasm_simulator'):
        backend = get_backend(backend_str)
        w = matrix_func(G)
        ops = get_qubitops(w)
        qaoa = QAOA(ops, optimizer, p, operator_mode='paulis')
        quantum_instance = QuantumInstance(backend)
        result = qaoa.run(quantum_instance)
        x = sample_most_likely(result['eigvecs'][0])
        return x

Credentials are already in use. The existing account in the session will be replaced.


In [25]:
import os
import sys
d = os.path.dirname(os.getcwd())+'/utils'
sys.path.append(d)
#from graphs import *
#from qubo_ising_generators import *
#from classical_solvers import *
import networkx as nx
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

def maximum_cut_qubo_dwave(G):
        Q = {}
        for a in list(G.edges()):
                Q[a] = 2
        for i in list(G.nodes()):
                Q[(i, i)] = -1*G.degree(i)
        return Q

def max_cut_qubo_matrix_ibmqx(G):
        qubo = maximum_cut_qubo_dwave(G)
        data = np.zeros((len(G), len(G)))
        for a in qubo:
                data[a[0], a[1]] = qubo[a]
                data[a[1], a[0]] = qubo[a]
        return data

def ibqmx_qaoa(G, optimizer, func, r):
        app_ratio = 0
        qubo_main = []
        counter = 0
        for avg in range(0, 100):
            counter += 1
            result_out = solve_ibmqx_ising_qubo(G, func, optimizer, r)
            result2 = []
            for i in result_out:
                  if i == 0:
                        result2.append(1)
                  if i == 1:
                        result2.append(0)
            x = max_cut_value(result2, G)
            qubo_main.append(x)
            app_ratio += x
            print(float(app_ratio)/float(counter))

print('maxcut')
out = []
for r in range(1, 20):
    G = nx.gnp_random_graph(7, 0.5, 101)
    res = ibqmx_qaoa(G, SLSQP(), max_cut_qubo_matrix_ibmqx, r)
    out.append(res)
    print(out)

maxcut


NameError: name 'Operator' is not defined